# OWL-ViT Object Detection Notebook

This notebook demonstrates how to run and visualize object detections on high-resolution satellite imagery using the `OwlViTPipeline` helper class.  

**Structure:**

1. **Imports & Initialization**  
   Import the pipeline class, create an instance for the chosen experiment (e.g. `baseline`), and configure any resize parameters.

2. **Load Data**  
   - Load & resize the input image.  
   - Load the textual prompts used for zero-shot detection.

3. **Run Experiments**  
   - **Multi-threshold run**: `run_threshold_experiments([…])` to save visualizations and collect metrics across several confidence cutoffs.  
   - **Single-threshold run**: `run_single_experiment(…)` for targeted analysis.  

4. **Visualization Helpers**  
   - **`show_only_prompt(results, prompt)`**: Plot detections for a specific prompt from pre-computed results.  
   - **`run_and_show_prompt(threshold, prompt)`**: Convenience one-liner for inference + prompt-specific plotting.  

5. **Export & Display Metrics**  
   - **GeoJSON export**: `run_and_save_geojson(threshold)` to save all detections as a GeoJSON file.  
   - **Metrics table**: `display_metrics()` to render a styled HTML table of detection counts and scores.  

Feel free to change the `experiment_name` or `resize_size` when instantiating the pipeline, and to tweak thresholds or prompt strings below.  


In [ ]:
# Setup
import sys
from pathlib import Path
import torch

tools_loc = Path.cwd().resolve().parents[2]
sys.path.insert(0, str(tools_loc))
              
from tools.owlvit_utils import OwlViTPipeline

In [ ]:
# Experiment configuration
experiment_name      = 'baseline'       # e.g., 'baseline', 'tiling', etc.
image_filename       = '<input_image_filename>'  # e.g., 'site_image.png'
prompts_file         = '<prompts_file>'          # e.g., 'prompts.txt'
resize_size          = (<width>, <height>)       # e.g., (1024, 1024)
thresholds           = [<threshold_1>, <threshold_2>, <threshold_3>, <threshold_4>]  # e.g., [0.001, 0.002, ...]


# Instantiate pipeline
pipeline = OwlViTPipeline(
    experiment_name=experiment_name,
    resize_size=resize_size
)

# Load original image & prompts
pipeline.load_image(image_filename)
pipeline.load_prompts(prompts_file)

In [ ]:
print(pipeline.prompts)

In [ ]:

# run detection at multiple thresholds (saves visuals and metrics)
metrics = pipeline.run_threshold_experiments(thresholds)

In [ ]:
# Loop over thresholds
for thresh in thresholds:

    # the raw results dict:
    inputs = pipeline.processor(
        text    = pipeline.prompts,
        images  = pipeline.image_resized,
        return_tensors = "pt"
    )
    with torch.no_grad():
        outputs = pipeline.model(**inputs)
    results = pipeline.processor.post_process_object_detection(
        outputs,
        target_sizes=torch.tensor([[pipeline.image_full.height,
                                    pipeline.image_full.width]]),
        threshold=thresh
    )[0]
    
    png_path = pipeline.save_visualisation(
        results,
        threshold=thresh
    )
    print("Saved visualization to:", png_path)
    
    # Save the per-threshold metrics JSON
    json_path = pipeline.save_metrics(
        results,
        threshold=thresh
    )
    print("Saved metrics to:", json_path)
    
    # save geojson
    geojson_path = pipeline.run_and_save_geojson(thresh)
    

In [ ]:
# run inference at threshold and display only detections for the specified prompt
for thresh in thresholds:
    pipeline.run_and_show_prompt(
        threshold=thresh,
        target_prompt="<some_prompt>"
    )
